# Paillier's encryption Scheme

The Paillier crypto system, invented by and named after Pascal Paillier in 1999, is a probabilistic asymmetric algorithm for public key cryptography. The problem of computing $n-th$ residue classes is believed to be computationally difficult. The decisional composite residuosity assumption is the intractability hypothesis upon which this cryptosystem is based. 

$$
\alpha +_E \beta = E(\alpha + \beta \ mod\ N)\\
x\times_E \alpha = E(x\alpha \ mod \ N)\\
\oplus_{i=1}^{t+1}a_i = a_1 +_E a_2 +_E \cdots +_E a_{t+1}
$$


## Key Generation:

1. Choose two large prime numbers $P$ and $Q$ randomly and indenpendently of each other such that $gcd(PQ, (P-1)(Q-1))=1$. This property is assured if both primes are qual lenth.

In [1]:
from klefki.const import (SECP256K1_P as P, SECP256K1_N as Q)

In [2]:
from math import gcd

assert gcd(P * Q, (P - 1) * (Q - 1)) == 1

2. Compute $N=PQ$ and $\lambda(N)=lcm(P-1, Q-1)$ be the Carmichael function of N.

In [3]:
from klefki.numbers import lcm

In [4]:
N = P * Q
Lam = lcm(P - 1, Q - 1)

3. Select random integer $\Gamma$ where $\Gamma \in \mathbf{Z}_{n^2}^*$.

In [5]:
from random import randint
from klefki.types.algebra.utils import randfield
from klefki.types.algebra.meta import field

F = field(N)
DF = field(N**2)
G = randfield(DF)

4. Ensure $n$ devides the order of $g$ by checking the existence of the following modular multiplicative inverse: 

$$
\mu = (L(G^{\lambda(N)}\mod N^2))^{-1} \mod \ N
$$

Where $L$ be a function defined over the set $\{x\in \mathbf{Z}_{N^2}: x=1 \ mod \ N\}$ computed as

$$
L(x) = \frac{x-1}{N}
$$

In [6]:
L = lambda x: (x - 1) // N
M = ~F(L(pow(G, Lam).value))

The public key is $(N, \Gamma)$ and the secret key is $(\lambda(N), \mu)$

If using p,q of equivalent length, a simpler variant of the above key generation steps would be to set $g = n + 1 , λ = φ ( n )$, and $\mu = φ(n)^{-1}\ mod \ n$, where $φ(n)=(p-1)(q-1)$

In [7]:
print("PrivKey: %s, %s" % (Lam, M))

PrivKey: 2234634654990432849929004166367641021238215826767191291571707378398254532167475902984296517123225539202576657105730699764493290075143829571044458305451072, 5907906290387101352271429657507557653391442009876775549000118650554725211448523147399026028057551205838436863740347547556115391592938522514435074169467368


In [8]:
print("PubKey: %s; %s" % (N, G))

PubKey: 13407807929942597099574024998205846127429294960603147749430244270389527193005087002084253735130200377185477318450090306135904455919285040173416176828872431; 47647907529450637011873503796044450568282215833048879663881211840970623061720641224079514294576642328201983852293137639614903818996937671800454967769063473185951737912373170887645639896766635355239009770435491747244287193555366271288766034496982445019683334996468542976690645769779486596886596356334275312423


## Encryption:

To Encrypt a message $M \in Z_N$, select $R \in_R Z_N^*$ and return $c=\Gamma^M R^N\ mod \ N^2$.

In [9]:
import random
import math
m = random.randint(0, N)
assert 0 <= m < N

r = random.randint(0, N)
assert 0 < r < N

assert math.gcd(r, N) == 1

In [10]:
pow(DF(r), N).value == pow(r, N, N**2)

True

In [11]:
c = G**m * DF(r)**N

In [12]:
print("Ciphtertext Text: %s" % c)

Ciphtertext Text: 136787907756669998335833932611373283248583773542855000674305975268186499491182520694445868032596683481751210015914698485042964892677101694982760320047643756387155474250080781572965916541670101835194286114700301768327971109134044620644958443248439267821946274607517806791037676822155992912225539507194748773858


## Decryption:

To decrypt a ciphertext $c \in Z_N$, let $L$ be a function defined over the set $\{u\in Z_{N^2}: u=1 \ mod \ N\}$ computed as $L(u) = \frac{u-1}{N}$. Then the decryption of $c$ is computed as

$$
\frac{L(c^{\lambda(N)}\ mod\ N^2)}{\mu}\ mod\ N
$$

In [13]:
assert F(m) == F(L(pow(DF(c), Lam).value)) * M

## Homomorphic

### Homomorphic addition

The product of two ciphertexts will decrypt to the sum of their corresponding plaintexts,
$$
D(E(m_1, r_1) \circ E(m_2, r_2)\ mod \ N^2) = m_1 \circ m_2 \ mod\ n
$$

## Ref

* Wikipedia: Pillier Cryptosystem https://en.wikipedia.org/wiki/Paillier_cryptosystem


* Encryption Performance Improvementsof the Paillier Cryptosystem https://eprint.iacr.org/2015/864.pdf


* Stackoverflow - Paillier algorithm encryption https://stackoverflow.com/questions/29217630/paillier-algorithm-encryption